# Decision Tree Classifier

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

### Helping Functions

In [2]:
def get_num_mistakes(y):
    '''Returns the number of mistakes in case of majority class label
    to all inputs. Also returns the majority label.
    '''
    
    y = y.flatten()
    label_counts = Counter(y)
    max_count = max(label_counts.values())
    # First majority class
    max_count_label = [label for label, count in label_counts.items() if 
                          count == max_count][0]
    num_mistakes = sum([count for label, count in label_counts.items() if 
                    label != max_count_label])
    return num_mistakes, max_count_label


def get_best_threshold(x, y):
    '''Returns the best threshold for the real valued input x with least 
    classification error on class labels y. Also returns classification error
    It binary classifies the data. For multiclass classification we can do 
    thresholding again and again.
    '''
    
    x = x.flatten()
    y = y.flatten()
    sorted_x = sorted(x)
    # No of Data Points
    m = x.shape[0]
    best_error = 2.0
    best_threshold = None
    
    for i in range(m - 1):
        
        threshold = (sorted_x[i] + sorted_x[i + 1]) / 2.0
        mistakes1 = 0
        mistakes2 = 0
        y1 = y[x < threshold]
        y2 = y[x >= threshold]

        if len(y1) > 0:

            mistakes1, _ = get_num_mistakes(y1)

        if len(y2) > 0:

            mistakes2, _ = get_num_mistakes(y2)

        error = (mistakes1 + mistakes2) / float(m)
        
        if error < best_error:
            
            best_error = error
            best_threshold = threshold
            
    return best_threshold, best_error
    
    
def get_best_splitting_feature(data, target, features_list, is_categorical_list):
    '''Returns the best feature with least classification error
    data : pandas dataframe containing training data
    target : target column name in data
    features_list : a list of column names used as features
    is_categorical_list : boolean list representing the corresponding columns
                          are categorical or not
    Also returns classification error and whether the feature is categorical.
    '''
    
    m = len(data)                       # No of data points
    best_feature = None
    is_categorical_best_feature = None
    best_error = 2.0

    for feature, is_categorical in zip(features_list, is_categorical_list):
        
        if is_categorical: # Categorical Feature
            
            num_mistakes = 0
            
            for category, frame in data.groupby(feature):
                
                num_mistakes += get_num_mistakes(np.array(frame[target]))[0]
                
            error = num_mistakes / float(m)
            
        else: # Continous Feature
            
            threshold, error = get_best_threshold(np.array(data[feature]),
                                                  np.array(data[target]))
            
        if error < best_error:
            
            best_error = error
            best_feature = feature
            is_categorical_best_feature = is_categorical
            
    return best_feature, best_error, is_categorical_best_feature

### The Model

In [3]:
# Attributes of the tree node
children_str = 'children'
splitting_feature_str = 'splitting_feature'
split_value_str = 'split_value'
label_str = 'label'
probability_str = 'probability'
is_categorical_str = 'is_categorical'
threshold_str = 'threshold'


def create_node(y):
    '''Creates a new node.
    label and probability attributes are set others are None
    use majority class label
    '''
    
    y = y.flatten()
    num_mistakes, label = get_num_mistakes(y)
    probability = 1 - (num_mistakes / float(len(y)))
    
    return {
        label_str: label,
        probability_str: probability,
        children_str: None,
        splitting_feature_str: None,
        split_value_str: None,
        is_categorical_str: None,
        threshold_str: None
    }



def create_decision_tree(data, target, features_list, is_categorical_list,
                        current_depth=0, max_depth=10):
    '''Recursively creates a new tree.
        data: A pandas dataframe object
        target: str representing target column
        features_list: list of str representing feature columns
        is_categorical_list: A boolean list of same size as features_list representing each feature
                             whether it is categorical or continous
    '''
    
    current_features = features_list[:]
    current_categorical_list = is_categorical_list[:]
    current_node = create_node(np.array(data[target]))
    
    if current_node[probability_str] == 1 or not current_features or current_depth >= max_depth:
        
        return current_node
    
    feature, error, is_categorical = get_best_splitting_feature(data, target, current_features, 
                                                               current_categorical_list)
    
    current_node[splitting_feature_str] = feature
    current_node[is_categorical_str] = is_categorical
    ind = current_features.index(feature)
    children = []
    
    if is_categorical:
        
        del current_categorical_list[ind]
        del current_features[ind]
        
        for category, frame in data.groupby(feature):
            
            child = create_decision_tree(frame, target, current_features, current_categorical_list, 
                                        current_depth + 1, max_depth)
            child[split_value_str] = category
            children.append(child)
    
    else:
        
        threshold, _ = get_best_threshold(np.array(data[feature]), np.array(data[target]))
        current_node[threshold_str] = threshold
        
        frame1 = data[data[feature] < threshold]
        frame2 = data[data[feature] >= threshold]
        child1 = None
        child2 = None

        if len(frame1) > 0:

            child1 = create_decision_tree(frame1, target, current_features, current_categorical_list,
                                          current_depth + 1, max_depth)

        if len(frame2) > 0:

            child2 = create_decision_tree(frame2, target, current_features, current_categorical_list,
                                          current_depth + 1, max_depth)
        
        children.append(child1)
        children.append(child2)
        
    current_node[children_str] = children
    
    return current_node


def classify(tree, x):
    '''Returns prediction for a single observation x
    '''
    
    if not tree[children_str]:
        
        return tree[label_str]
    
    feature = tree[splitting_feature_str]
    is_categorical = tree[is_categorical_str]
    
    if is_categorical:
        
        for child in tree[children_str]:
            
            if child[split_value_str] == x[feature]:
                
                return classify(child, x)
            
    else:
        
        threshold = tree[threshold_str]
        children = tree[children_str]
        
        if x[feature] < threshold:
            
            return classify(children[0], x)
        
        else:
            
            return classify(children[1], x)
        
        

def display_tree(tree, is_categorical_parent = True, level=0):
    '''Recursively displays the tree node by node
    '''
    
    if tree:

        print('\t' * level, end='')

        if is_categorical_parent:

            print(tree[split_value_str], '\b:', end=' ')

        else:

            print(tree[threshold_str], '\b:', end=' ')

        print(tree[splitting_feature_str], '->', tree[label_str])

        if not tree[children_str]:
            return

        for child in tree[children_str]:
            display_tree(child, tree[is_categorical_str], level + 1)


def accuracy(tree, data, target):
    '''Returns accuracy of the model on the data
    data: A pandas dataframe object
    '''
    
    prediction = data.apply(lambda x: classify(tree, x), axis = 1)

    return 100.0 * np.sum(data[target] == prediction) / float(len(data))

### Categorical Data

In [9]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'bad_loans'

In [10]:
df = pd.read_csv('data/lending-club-data.csv', usecols=features + [target])
df.head()

,term,grade,emp_length,home_ownership,bad_loans
0,36 months,B,10+ years,RENT,0
1,60 months,C,< 1 year,RENT,1
2,36 months,C,10+ years,RENT,0
3,36 months,C,10+ years,RENT,0
4,36 months,A,3 years,RENT,0


#### Train Test Split

In [13]:
train_size = int(0.8*len(df))
random_indices = np.random.permutation(len(df))

df_train = df[:train_size]
df_test = df[train_size:]

print(len(df_train), len(df_test))

98085 24522


#### Building Tree

In [14]:
is_cat_list = [True] * 4
tree = create_decision_tree(df_train, target, features, is_cat_list)

#### Test Set Performance

In [17]:
accuracy(tree, df_test, target)

81.8978876111247

### Continous Data

In [18]:
features = ['sepal length', 'sepal width', 'petal length', 'petal width']
target = 'label'

In [20]:
df = pd.read_csv('data/iris.csv', header=None, names=features + ['class'])
class_to_label_dict = {'Iris-setosa': 0, 'Iris-versicolor': 1, 
                      'Iris-virginica': 2}
df['label'] = df['class'].map(lambda x: class_to_label_dict[x])

df.iloc[np.random.randint(0, len(df), 10)]

,sepal length,sepal width,petal length,petal width,class,label
58,6.6,2.9,4.6,1.3,Iris-versicolor,1
62,6.0,2.2,4.0,1.0,Iris-versicolor,1
45,4.8,3.0,1.4,0.3,Iris-setosa,0
121,5.6,2.8,4.9,2.0,Iris-virginica,2
87,6.3,2.3,4.4,1.3,Iris-versicolor,1
8,4.4,2.9,1.4,0.2,Iris-setosa,0
148,6.2,3.4,5.4,2.3,Iris-virginica,2
126,6.2,2.8,4.8,1.8,Iris-virginica,2
109,7.2,3.6,6.1,2.5,Iris-virginica,2
104,6.5,3.0,5.8,2.2,Iris-virginica,2


#### Train Test Split

In [21]:
train_size = int(0.8*len(df))
random_indices = np.random.permutation(len(df))

df_train = df[:train_size]
df_test = df[train_size:]

print(len(df_train), len(df_test))

120 30


#### Building Tree

In [28]:
is_cat_list = [False] * 4
tree = create_decision_tree(df_train, target, features, is_cat_list, max_depth=2)

#### Test Set Performance

In [30]:
accuracy(tree, df_test, 'label')

83.33333333333333

We can also use a mixture of continous and categorical features. The model will handle them itself.